# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ancud,-41.8697,-73.8203,51.21,71,95,10.36,CL,1681692028
1,1,mataura,-46.1927,168.8643,65.82,49,96,3.22,NZ,1681692029
2,2,la ronge,55.1001,-105.2842,55.65,24,20,9.22,CA,1681692030
3,3,avarua,-21.2078,-159.7750,75.25,53,100,5.75,CK,1681692030
4,4,magadan,59.5667,150.8000,33.53,87,73,4.23,RU,1681692031


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
city_map= city_data_df.hvplot.points(
    x= 'Lng', y='Lat', geo=True, hover_cols=['City', 'Country', 'Humidity'],
    size='Humidity', size_scale=0.5, width=800, height=600, color='blue'
)

# Display the map
hvplot.show(city_map)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_cities_df.dropna()

# Display sample data
display(ideal_cities_df)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
251,251,bagdarin,54.4333,113.6,22.66,53,0,4.36,RU,1681692098
396,396,provideniya,64.3833,-173.3,24.91,50,0,0.00,RU,1681692126


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
display(hotel_df)


,City,Country,Lat,Lng,Humidity,Hotel Name
251,bagdarin,RU,54.4333,113.6,53,
396,provideniya,RU,64.3833,-173.3,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apikey": geoapify_key,
    "type": "hotel", 
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:${lng}, ${lat}, ${radius}"
    params["bias"] = f"proximity:${lng}, ${lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mindyak - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
487,mindyak,RU,54.0231,58.7891,38,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x='Lng', y='Lat', geo=True, hover_cols=['City', 'Country', 'Hotel Name'],
    size='Humidity', size_scale=0.5, width=800, height=600, color='blue', 
    title="Ideal Cities and Nearby Hotels"
)

# Display the map
hvplot.show(hotel_map)